In [5]:
import sklearn
import numpy as np
from matplotlib import pyplot as plt
import pandas as pan

## Importing training data

In [42]:
training_data = pan.read_csv('train_data.tsv',sep='\t')
print(training_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6333 entries, 0 to 6332
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   6333 non-null   int64 
 1   text_a  6333 non-null   object
dtypes: int64(1), object(1)
memory usage: 99.1+ KB
None


## Cleaning the data

In [41]:
import re
class cleaner:
   def __init__(self):
      self.links = re.compile(r'http\S+')
      self.amp = re.compile(r'[^,]*&amp[^,]*')
   def clean_data(self, data):
      data = self.links.sub('', data)
      s = data.encode('ascii',errors='ignore')
      data = s.decode()
      data = self.amp.sub('', data)
      return data

In [43]:
mr_clean = cleaner()
training_data['text_a'] = training_data['text_a'].apply(mr_clean.clean_data)
del mr_clean

### Quick analysis

In [53]:
import collections
import string
import nltk
class counter:
    def __init__(self):
        self.word_count = collections.Counter()
    def count(self, data):
        words = data.split(' ')
        for word in words:
            self.word_count[word] += 1
        self.word_count[''] = 0
    def get(self, n):
        return self.word_count.most_common(n)
    def get_all(self):
        return self.word_count.most_common()

In [54]:
mr_count = counter()
training_data['text_a'].apply(mr_count.count)
print(mr_count.get_all())

[('the', 5508), ('of', 4087), ('to', 3597), ('in', 3436), ('and', 2537), ('a', 2477), ('is', 1887), ('for', 1483), ('are', 1361), ('cases', 1317), ('that', 1194), ('COVID-19', 1135), ('#COVID19', 1036), ('on', 1030), ('from', 1003), ('have', 939), ('The', 905), ('with', 875), ('has', 842), ('new', 702), ('at', 662), ('coronavirus', 638), ('as', 620), ('people', 594), ('be', 584), ('by', 574), ('been', 565), ('tests', 545), ('not', 524), ('number', 482), ('more', 475), ('confirmed', 455), ('we', 448), ('this', 434), ('was', 423), ('deaths', 421), ('A', 421), ('can', 420), ('reported', 415), ('will', 411), ('it', 409), ('than', 396), ('total', 387), ('#coronavirus', 366), ('our', 363), ('you', 356), ('or', 347), ('who', 328), ('all', 322), ('now', 321), ('New', 320), ('an', 311), ('India', 311), ('testing', 306), ('We', 292), ('which', 280), ('your', 279), ('There', 276), ('about', 272), ('COVID-19.', 271), ('there', 269), ('but', 268), ('they', 267), ('no', 267), ('states', 262), ('one'